Определение языка (language detection)
--------------------

* **Множество случаев** — тексты на разных языках
* **Множество классов** — языки

### Первый метод: частотные слова

Очень простой и неплохой по качеству метод. Сначала создаем частотный словарь для всех языков и берем самые частотные слова. После этого для каждого текста, который нам надо расклассифицировать, смотрим, частотных слов какого языка в нем больше - тот язык и выбираем.

Метод неплохо работает на текстах длиннее 50 слов и быстро имлементируется. 

В качестве корпусов и текстов для тестирования будем использовать статьи Википедии на разных языках. Скачать Википедию можно различными способами:

* Дампы википедии: https://dumps.wikimedia.org/backup-index.html

* wikiextractor: http://medialab.di.unipi.it/wiki/Wikipedia_Extractor

* annotated_wikiextractor: https://github.com/jodaiber/Annotated-WikiExtractor

* wikipedia: https://pypi.python.org/pypi/wikipedia/

#### Скачаем немного википедии для тестов
Воспользуемся пакетом *wikipedia*:

`pip install wikipedia`

In [141]:
def get_texts_for_lang(lang, n=10): # функция для скачивания статей из википедии
    wikipedia.set_lang(lang)
    wiki_content = []
    pages = wikipedia.random(n)
    for page_name in pages:
        try:
            page = wikipedia.page(page_name)
        except wikipedia.exceptions.WikipediaException:
            print('Skipping page {}'.format(page_name))
            continue

        wiki_content.append('{}\n{}'.format(page.title, page.content.replace('==', '')))

    return wiki_content

In [142]:
import wikipedia # скачиваем по 100 статей для каждого языка. Это может занять какое-то время (5-10 минут. как правило)

wiki_texts = {}
for lang in ('kk', 'uk', 'de', 'fr'): # казахский в википедии — это kk,
                                      # украинский — uk
    wiki_texts[lang] = get_texts_for_lang(lang, 100)
    #print(lang, len(wiki_texts[lang]))

/Users/Valeriya/anaconda/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Users/Valeriya/anaconda/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Skipping page Құпия экспедиция
Skipping page Борисоглібський монастир
Skipping page Сабле (значення)
Skipping page Слов'янськ (значення)
Skipping page Орлова
Skipping page Leisching
Skipping page Grąbkowo
Skipping page Rudnicki
Skipping page Knaap
Skipping page Dirtl
Skipping page Kartal
Skipping page Brugger
Skipping page Johann Franz
Skipping page Gravesville
Skipping page Nowy Majdan
Skipping page Baier
Skipping page Soldiers' and Sailors' Monument
Skipping page Cambria
Skipping page Robustel
Skipping page Château de Reux (homonymie)
Skipping page Hopkinton
Skipping page Tenaille (homonymie)
Skipping page Priez (homonymie)


In [143]:
# print(wiki_texts['kk'][0]) # распечатаем пару текстов, чтобы убедиться, что все хорошо
# print(wiki_texts['de'][0])

#### Считаем частотный список примерно так:

In [144]:
import codecs
import collections
import sys

def tokenize(text):
    return text.split(' ')



freqs_kk = collections.defaultdict(lambda: 0)
corpus_kk = wiki_texts['kk']
for article in corpus_kk:
    for word in tokenize(article.replace('\n', '').lower()):
        freqs_kk[word] += 1

        
freqs_uk = collections.defaultdict(lambda: 0)
corpus_uk = wiki_texts['uk']
for article in corpus_uk:
    for word in tokenize(article.replace('\n', '').lower()):
        freqs_uk[word] += 1
        

        
freqs_de = collections.defaultdict(lambda: 0)
corpus_de = wiki_texts['de']
for article in corpus_de:
    for word in tokenize(article.replace('\n', '').lower()):
        freqs_de[word] += 1
        
        
freqs_fr = collections.defaultdict(lambda: 0)
corpus_fr = wiki_texts['fr']
for article in corpus_fr:
    for word in tokenize(article.replace('\n', '').lower()):
        freqs_fr[word] += 1

        
        
# for word in sorted(freqs, key=lambda w: freqs[w], reverse=True)[:50]:
#     print('{}\t{}'.format(freqs[word], word))

In [145]:
set_dict_kk = set()
for key in sorted(freqs_kk, key=lambda w: freqs_kk[w], reverse=True):
        if key not in freqs_de and key not in freqs_fr and key not in freqs_uk:
            set_dict_kk.add(key)
        if len(set_dict_kk) > 499:
            break


In [146]:
set_dict_de = set()
for key in sorted(freqs_de, key=lambda w: freqs_de[w], reverse=True):
        if key not in freqs_kk and key not in freqs_fr and key not in freqs_uk:
            set_dict_de.add(key)
        if len(set_dict_de) > 499:
            break

In [147]:
set_dict_uk = set()
for key in sorted(freqs_uk, key=lambda w: freqs_uk[w], reverse=True):
        if key not in freqs_kk and key not in freqs_fr and key not in freqs_de:
            set_dict_uk.add(key)
        if len(set_dict_uk) > 499:
            break

In [148]:
set_dict_fr = set()
for key in sorted(freqs_fr, key=lambda w: freqs_fr[w], reverse=True):
        if key not in freqs_kk and key not in freqs_uk and key not in freqs_de:
            set_dict_fr.add(key)
        if len(set_dict_fr) > 499:
            break

## Генерируем новые тексты для теста

In [149]:
#import wikipedia # скачиваем по 100 статей для каждого языка. Это может занять какое-то время (5-10 минут. как правило)

wiki_texts_test = {}
for lang in ('kk', 'uk', 'de', 'fr'): # казахский в википедии — это kk,
                                      # украинский — uk
    wiki_texts_test[lang] = get_texts_for_lang(lang, 100)

/Users/Valeriya/anaconda/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Users/Valeriya/anaconda/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Skipping page Сарыағаш (айрық)
Skipping page Шеол
Skipping page Валя-Лунге
Skipping page Ель-Міраж
Skipping page Крючков
Skipping page Подільський провулок
Skipping page Walworth
Skipping page CNB
Skipping page Bodleian
Skipping page Rich Creek
Skipping page Low-Entry
Skipping page John Bradford
Skipping page Jim Smith
Skipping page Lämmerbichl
Skipping page Jakobeit
Skipping page Walcha
Skipping page Tavera (Begriffsklärung)
Skipping page Frappa
Skipping page Marthaler
Skipping page Michael Geyer
Skipping page Schneider-Esleben
Skipping page Enki
Skipping page Reille
Skipping page Inversion
Skipping page CPI
Skipping page Courtier (homonymie)


In [211]:
num_of_mistakes = 0
for i in wiki_texts_test:
    for ind in range(len(wiki_texts_test[i])-1):
        set_text = set(tokenize(wiki_texts_test[i][ind]))
        lengths = []  # kk, de, fr, uk
        set_inter_kk = len(set_text & set_dict_kk)
        lengths.append(set_inter_kk)
        set_inter_de = len(set_text & set_dict_de)
        lengths.append(set_inter_de)
        set_inter_fr = len(set_text & set_dict_fr)
        lengths.append(set_inter_fr)
        set_inter_uk = len(set_text & set_dict_uk)
        lengths.append(set_inter_uk)
        max_num = max(lengths)
        if max_num == lengths[0]:
            predicted = 'kk'
        elif max_num == lengths[1]:
            predicted = 'de'
        elif max_num == lengths[2]:
            predicted = 'fr'
        else:
            predicted = 'uk'
        if predicted != i:
            num_of_mistakes += 1
        

In [212]:
sum_all = 0
for i in wiki_texts_test:
    sum_all += len(wiki_texts_test[i])

In [213]:
print((num_of_mistakes)/sum_all) # количество ошибок

0.0


#### Нужно сделать это для каждого языка и отфильтровать повторяющиеся

Теперь можно загружать готовые частотные словари и классифицировать тексты, просто считая количество слов в каждом.

### Второй метод: частотные символьные n-граммы

Создадим функцию, которая преобразовывает строку в массив n-грамм заданной длины.

In [159]:
from itertools import islice, tee

def make_ngrams(text):
    N = 3 # задаем длину n-граммы
    ngrams = zip(*(islice(seq, index, None) for index, seq in enumerate(tee(text, N))))
    ngrams = [''.join(x) for x in ngrams]
    return ngrams

Теперь создадим частотные словари n-грамм аналогично первому методу.

In [194]:
n_freqs_fr = collections.defaultdict(lambda: 0)
corpus_fr = wiki_texts['fr']
for article in corpus_fr:
    for ngram in make_ngrams(article.replace('\n', '').lower()):
        n_freqs_fr[ngram] += 1


n_freqs_de = collections.defaultdict(lambda: 0)
corpus_de = wiki_texts['de']
for article in corpus_de:
    for ngram in make_ngrams(article.replace('\n', '').lower()):
        n_freqs_de[ngram] += 1       
        
        
n_freqs_uk = collections.defaultdict(lambda: 0)
corpus_uk = wiki_texts['uk']
for article in corpus_uk:
    for ngram in make_ngrams(article.replace('\n', '').lower()):
        n_freqs_uk[ngram] += 1       
        
        
n_freqs_kk = collections.defaultdict(lambda: 0)
corpus_kk = wiki_texts['kk']
for article in corpus_kk:
    for ngram in make_ngrams(article.replace('\n', '').lower()):
        n_freqs_kk[ngram] += 1         
        
        
# for ngram in sorted(freqs, key=lambda n: freqs[n], reverse=True)[:50]:
#     print('{}\t{}'.format(freqs[ngram], ngram))

In [195]:
ngr_kk = set()
for key in sorted(n_freqs_kk, key=lambda w: n_freqs_kk[w], reverse=True):
        if key not in n_freqs_de and key not in n_freqs_fr and key not in n_freqs_uk:
            ngr_kk.add(key)
        if len(ngr_kk) > 499:
            break

ngr_de = set()
for key in sorted(n_freqs_de, key=lambda w: n_freqs_de[w], reverse=True):
        if key not in n_freqs_kk and key not in n_freqs_fr and key not in n_freqs_uk:
            ngr_de.add(key)
        if len(ngr_de) > 499:
            break
            
ngr_uk = set()
for key in sorted(n_freqs_uk, key=lambda w: n_freqs_uk[w], reverse=True):
        if key not in n_freqs_kk and key not in n_freqs_fr and key not in n_freqs_de:
            ngr_uk.add(key)
        if len(ngr_uk) > 499:
            break

ngr_fr = set()
for key in sorted(n_freqs_fr, key=lambda w: n_freqs_fr[w], reverse=True):
        if key not in n_freqs_kk and key not in n_freqs_uk and key not in n_freqs_de:
            ngr_fr.add(key)
        if len(ngr_fr) > 499:
            break

In [207]:
num_of_mistakes_ngr = 0
for i in wiki_texts_test:
    for ind in range(len(wiki_texts_test[i])-1):
        set_text = set(make_ngrams(wiki_texts_test[i][ind]))
        lengths = []  # kk, de, fr, uk
        set_inter_kk = len(set_text & ngr_kk)
        lengths.append(set_inter_kk)
        set_inter_de = len(set_text & ngr_de)
        lengths.append(set_inter_de)
        set_inter_fr = len(set_text & ngr_fr)
        lengths.append(set_inter_fr)
        set_inter_uk = len(set_text & ngr_uk)
        lengths.append(set_inter_uk)
        max_num = max(lengths)
        if max_num == lengths[0]:
            predicted = 'kk'
        elif max_num == lengths[1]:
            predicted = 'de'
        elif max_num == lengths[2]:
            predicted = 'fr'
        else:
            predicted = 'uk'
        if predicted != i:
            num_of_mistakes += 1
        #print(i, predicted)

In [200]:
print((num_of_mistakes_ngr)/sum_all)

0.0


# Функции для определения языка

In [215]:
def dictionary(text):
    set_text = set(tokenize(text))
    lengths = []  # kk, de, fr, uk
    set_inter_kk = len(set_text & set_dict_kk)
    lengths.append(set_inter_kk)
    set_inter_de = len(set_text & set_dict_de)
    lengths.append(set_inter_de)
    set_inter_fr = len(set_text & set_dict_fr)
    lengths.append(set_inter_fr)
    set_inter_uk = len(set_text & set_dict_uk)
    lengths.append(set_inter_uk)
    max_num = max(lengths)
    if max_num == lengths[0]:
        predicted = 'kk'
    elif max_num == lengths[1]:
        predicted = 'de'
    elif max_num == lengths[2]:
        predicted = 'fr'
    else:
        predicted = 'uk'
    return predicted

In [214]:
def ngrams(text):
    set_text = set(make_ngrams(text))
    lengths = []  # kk, de, fr, uk
    set_inter_kk = len(set_text & ngr_kk)
    lengths.append(set_inter_kk)
    set_inter_de = len(set_text & ngr_de)
    lengths.append(set_inter_de)
    set_inter_fr = len(set_text & ngr_fr)
    lengths.append(set_inter_fr)
    set_inter_uk = len(set_text & ngr_uk)
    lengths.append(set_inter_uk)
    max_num = max(lengths)
    if max_num == lengths[0]:
        predicted = 'kk'
    elif max_num == lengths[1]:
        predicted = 'de'
    elif max_num == lengths[2]:
        predicted = 'fr'
    else:
        predicted = 'uk'
    return predicted

In [203]:
def what_language(text):
    a1 = dictionary(text)
    a2 = ngrams(text)
    return a1, a2

In [181]:
text = 'Пес сві́йський або сві́йський соба́ка (Canis lupus familiaris або Canis familiaris) — культигенна тварина. Термін застосовують як для домашніх, так і для бездомних тварин. Свійський пес був одним з найбільш широко застосовуваних службових та компанійських тварин протягом всієї історії людства. За різними оцінками, одомашнення вовка відбулося від 100 000 до 15 000 років тому. mDNA тестування показує, що еволюційний розкол між собаками і вовками відбувся близько 100 000 років тому. Собака швидко став незамінним у всіх світових культурах та був дуже цінним в ранніх людських поселеннях. Зокрема вважають, що успішна еміграція через Берингову протоку могла б бути неможливою без їздових собак. Собаки виконують багато видів робіт для людей, таких як полювання, охорона, служба в поліції та військах, а також собаки допомагають пасти стада худоби, допомагають інвалідам та служать компанійськими сімейними собаками. Ця універсальність, більша ніж практично в будь-якої іншої відомої людству тварини, дала собаці прізвисько «найкращий друг людини». За підрахунками, на планеті на 2015 рік проживало близько 525 мільйонів собак[1]. Завдяки селекції, було розведено сотні різноманітних порід, та зараз виявляють більше поведінкових та морфологічних відмінностей між собаками різних порід, ніж в будь-яких інших наземних ссавців. Наприклад, висота в холці може варіювати від кількох сантиметрів (чивава) до майже метра (ірландський вольфгаунд, великий данець); забарвлення — від білого до чорного, включаючи світло-жовте, сіре, коричневе з великим розмаїттям відтінків.'

In [182]:
text_2 = 'Der Haushund (Canis lupus familiaris) ist ein Haustier und wird als Heim- und Nutztier gehalten. Seine wilde Stammform ist der Wolf, dem er als Unterart zugeordnet wird. Wann die Domestizierung stattfand ist umstritten; wissenschaftliche Schätzungen variieren zwischen 15.000 und 100.000 Jahren vor unserer Zeit. Im engeren Sinn bezeichnet man als Haushund die Hunde, die überwiegend im Haus gehalten werden, und kennzeichnet damit also eine Haltungsform. Historisch wurde ein Hund, der zur Bewachung des Hauses gehalten wird, als Haushund bezeichnet.[1] Eine weitere Verwendung des Begriffs ist die Einschränkung auf sozialisierte (Haus-)Hunde, also Hunde, die an das Zusammenleben mit Menschen in der menschlichen Gesellschaft gewöhnt und an dieses angepasst sind. Damit wird der Haushund abgegrenzt gegen wild lebende, verwilderte oder streunende Hunde, die zwar auch domestiziert, aber nicht sozialisiert sind.[2] Der Dingo ist ebenfalls ein Haushund, wird jedoch provisorisch als eigenständige Unterart des Wolfes geführt.[3]'

In [209]:
print(what_language(text))

(('uk', [0, 0, 0, 34]), ('uk', [0, 0, 0, 167]))


In [210]:
print(what_language(text_2))

(('de', [0, 25, 0, 0]), ('de', [0, 41, 0, 0]))
